In [1]:
import torch
from torch import nn

from skimage import data

## Pooling

Documentação: https://pytorch.org/docs/stable/nn.html#torch.nn.MaxPool2d

```python
torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
```

**```kernel_size```** <br>
Tamanho dos *Field of View*. Pode ser uma tupla ou um único número. Ex: ```kernel_size = 3``` definirá FoV de $3 \times 3$

**```stride```** <br>
Controla o pulo da janela deslizante. 

**```padding```** <br>
Preenchimento com zeros nas bordas da imagem.

A camda de pooling espera uma entrada de **pelo menos** 3 dimensões ($C \times H \times W$), mas em geral a rede irá prover também a dimensão do batch ($B \times C \times H \times W$) 

In [8]:
tensor = torch.Tensor([[[1,2,3], 
                        [4,5,6], 
                        [7,8,9]]])
pool = nn.MaxPool2d(2, stride = 1)

saida = pool(tensor)
print(tensor.size())
print(tensor)
print(saida.size())
print(saida)

torch.Size([1, 3, 3])
tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])
torch.Size([1, 2, 2])
tensor([[[5., 6.],
         [8., 9.]]])


Ao processar dados com múltiplos canais, a camada de pooling processa cada canal de entrada separadamente ao invés de processar todos os canais como em uma camada convolucional. Isso significa que **o número de canais de saída para a camada de pooling é o mesmo que o número de canais de entrada**. 

Vamos processar abaixo a imagem da astronauta.

In [11]:
conv = nn.Conv2d(in_channels=3, out_channels=16, 
                 kernel_size=3, padding=1)

rgb = data.astronaut()
rgb_tns = torch.Tensor(rgb)
rgb_tns = rgb_tns.permute(2, 0, 1).unsqueeze(0)
mapa_de_ativacao = conv(rgb_tns)
print('Feature Map:', mapa_de_ativacao.shape)

Feature Map: torch.Size([1, 16, 512, 512])


In [14]:
pool = nn.MaxPool2d(kernel_size=2)
saida = pool(mapa_de_ativacao)
print(saida.size())

torch.Size([1, 16, 256, 256])


## Batch Normalization

Documentação: https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm2d

```python
torch.nn.BatchNorm2d(num_features)
```

**```num_features```**<br>
$\mathbf{\gamma}$ e $\mathbf{\beta}$ são aprendidos individualmente para cada canal da entrada. Em ativações de camadas intermediárias, esse valor corresponde ao **número de feature maps**. 


In [20]:
blococonv = nn.Sequential(

    nn.Conv2d(3, 32, kernel_size = 3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=10)

)

print(blococonv)

print(rgb_tns.size())
saida = blococonv(rgb_tns)
print(saida.size())

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([1, 3, 512, 512])
torch.Size([1, 32, 51, 51])
